In [ ]:
# 1. Imports
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

import joblib

In [ ]:
# 2. Coleta dos dados
symbol = 'AAPL'
start_date = '2018-01-01'
end_date = '2024-07-20'

df = yf.download(symbol, start=start_date, end=end_date)
df = df[['Close']]

In [ ]:
# 3. Pré-processamento
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

window_size = 60

X = []
y = []

for i in range(window_size, len(scaled_data)):
    X.append(scaled_data[i - window_size:i])
    y.append(scaled_data[i])

X, y = np.array(X), np.array(y)


In [ ]:
# 4. Separação dos dados
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
# 5. Modelo LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

In [ ]:
# 6. Avaliação
y_pred = model.predict(X_test)

y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}")

In [ ]:
# 7. Visualização
plt.figure(figsize=(14,6))
plt.plot(y_test_inv, label="Real")
plt.plot(y_pred_inv, label="Previsto")
plt.title("Previsão com LSTM")
plt.xlabel("Dias")
plt.ylabel("Preço")
plt.legend()
plt.show()

In [ ]:
# 8. Salvamento do modelo e scaler
model.save("../app/model/lstm_model.keras")
joblib.dump(scaler, "../app/model/scaler.gz")
